In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k


In [15]:
data = pd.read_csv('../input/homework02/retail_train.csv')


In [16]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

**Задание 1. Weighted Random Recommendation**

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

* Можно сэмплировать товары случайно, но пропорционально какому-либо весу
* Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [17]:
def weighted_random_recommendation(items_weights, n=5):

    df = items_weights.copy()
    df = df.sort_values('weight',ascending=False)
    r = np.random.randint(n,len(items_weights))
    df= df.iloc[:r]
    recs = np.random.choice(df.iloc[:r,0],size = n,replace=False)
    
    return recs.tolist()

In [18]:
items_weights = data.groupby('item_id')['quantity'].sum().reset_index()


In [19]:
items_weights['weight'] =  np.log(items_weights.quantity-0.001)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('quantity',axis=1,inplace=True)


In [20]:
items_weights.weight.sum()

In [21]:
weighted_random_recommendation(items_weights)


**Задание 2. Расчет метрик**
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [22]:
result = pd.read_csv('../input/homework02/retail_train.csv',dtype=np.object)# загрузка predict с семианара
for i in result:
    result[i] = result[i].apply(lambda x: eval(x))
    print(i,type(result[i][0]))
    
    

In [ ]:
result['weighted_random'] = result.user_id.apply(lambda x: weighted_random_recommendation(items_weights))